In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier

In [6]:
#Initmessage
print ("starting Regression") 

#lade von CSV
data_frame_investigated = pd.read_csv("Data/crypto-markets.csv", sep="," )


print("data collected")

print(data_frame_investigated.head())

starting Regression
data collected
      slug symbol     name        date  ranknow    open    high     low  \
0  bitcoin    BTC  Bitcoin  2013-04-28        1  135.30  135.98  132.10   
1  bitcoin    BTC  Bitcoin  2013-04-29        1  134.44  147.49  134.00   
2  bitcoin    BTC  Bitcoin  2013-04-30        1  144.00  146.93  134.05   
3  bitcoin    BTC  Bitcoin  2013-05-01        1  139.00  139.89  107.72   
4  bitcoin    BTC  Bitcoin  2013-05-02        1  116.38  125.60   92.28   

    close  volume        market  close_ratio  spread  
0  134.21     0.0  1.500520e+09       0.5438    3.88  
1  144.54     0.0  1.491160e+09       0.7813   13.49  
2  139.00     0.0  1.597780e+09       0.3843   12.88  
3  116.99     0.0  1.542820e+09       0.2882   32.17  
4  105.21     0.0  1.292190e+09       0.3881   33.32  


In [7]:
#eliminiere redundante Attribute
REDUNDANT_ATTRIBUTES=['symbol','name'] 
data_frame_investigated.drop(REDUNDANT_ATTRIBUTES,axis=1,inplace=True) #loeschen aus Dataframe

#set Date as Index


print(data_frame_investigated.head())

      slug        date  ranknow    open    high     low   close  volume  \
0  bitcoin  2013-04-28        1  135.30  135.98  132.10  134.21     0.0   
1  bitcoin  2013-04-29        1  134.44  147.49  134.00  144.54     0.0   
2  bitcoin  2013-04-30        1  144.00  146.93  134.05  139.00     0.0   
3  bitcoin  2013-05-01        1  139.00  139.89  107.72  116.99     0.0   
4  bitcoin  2013-05-02        1  116.38  125.60   92.28  105.21     0.0   

         market  close_ratio  spread  
0  1.500520e+09       0.5438    3.88  
1  1.491160e+09       0.7813   13.49  
2  1.597780e+09       0.3843   12.88  
3  1.542820e+09       0.2882   32.17  
4  1.292190e+09       0.3881   33.32  


In [8]:
# bestimme Trends

df = data_frame_investigated['close'].shift(periods=-1).to_frame()
data_frame_investigated['close_s'] = df

data_frame_investigated = data_frame_investigated[data_frame_investigated['slug'] == 'bitcoin']
data_frame_investigated['trend'] = 5

for i, r in data_frame_investigated.iterrows():
    
    if r["close"] < r["close_s"]:
        #Kurs steigt anschließend
        #data_frame_investigated = data_frame_investigated.set_value(index =i, col = ['trend'], value = 1)
        data_frame_investigated.at[i, 'trend'] = 1
    else:
        #Kurs fällt anschließend
        #data_frame_investigated = data_frame_investigated.set_value(index =i, col = ['trend'], value = 0)  
        data_frame_investigated.at[i, 'trend'] = 0
        
#Y Wert für Klassifizierer -> Trendwerte als eigene Spalte
data_frame_trend = data_frame_investigated['trend'].to_frame() 


In [9]:
np.size(data_frame_trend)

1793

In [33]:
from sklearn.model_selection import train_test_split #library fuer Split
from sklearn import linear_model #lineare Regression
import math #import fuer msqError
from sklearn import metrics 
from sklearn.metrics import roc_curve
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix


#dependency :patsy

def cryptoclassifier(crypto_id, feature_to_compare,crypto_id_feature_name="slug", documentMode=True):
    """
    Classifier zur Bestimmung eines Features anhand gleichen Features der anderen Cryptos
    """
    #Cryptos Gruppieren
    
    #ich kann nur anhand von Tagen schliessen, an denen die Crypto Existiert
    #erstmal alle Tage holen an denen die Crypto gehandelt wird
    dates_of_crypto_trade= data_frame_investigated[
        data_frame_investigated[crypto_id_feature_name] == crypto_id
    ]['date']
    
   
    #print(dates_of_crypto_trade.head())
    
    #dann die Tupel Filtern, nur die Tage an denen Crypto getraded wird uebernehmen,
    #nur compare feature uebernehmen
    data_frame_id_exists= data_frame_investigated[
        data_frame_investigated['date'].isin(dates_of_crypto_trade)
    ][['date',crypto_id_feature_name,feature_to_compare]]
    
    #daten Pivotieren so dass Cryptos zu columns werden
    data_frame_id_exists= data_frame_id_exists.pivot(
        index='date', columns=crypto_id_feature_name, values=feature_to_compare
        ).reset_index()
    
 
    
    data_frame_id_exists=data_frame_id_exists.fillna(value=0)
    data_frame_id_exists=data_frame_id_exists.dropna()
    
   
    #date rausloeschen
    data_frame_id_exists.drop('date',axis=1,inplace=True)
    #print("cleaned Frame")
    #print(data_frame_id_exists.head())
    data_frame_for_predictions = data_frame_id_exists #copy data_frame_id_exists just for safety purposes
    
    #print("Größe data_frame_for_predictions:", np.size(data_frame_for_predictions))
    
    ###############Train Test Split:
    Y = data_frame_trend.values.ravel() #changes Y from collumn to 1d array
    
    #print(np.size(Y)) #check size of Y
    
    
    feature_train, feature_test, y_train, y_test = train_test_split (
        data_frame_id_exists,
        Y,
        test_size=0.1
    )

    ##############Classifier machen
    classifier_model = AdaBoostClassifier(
        base_estimator=None, learning_rate=0.5, n_estimators=100, random_state=None)



        #Classifier fitten
    classifier_model.fit(feature_train, y_train)
    predictions = classifier_model.predict(data_frame_for_predictions)
    #Score_train = classifier_model.score(feature_train,y_train) #score test score value
    Score_test = classifier_model.score(feature_test,y_test) #model score value
    Confusion_matrix = confusion_matrix(Y,predictions) #returns confusion matrix
    fpr, tpr, thresholds = roc_curve(Y,predictions, pos_label=None, sample_weight=None, drop_intermediate=True)
                                    
    #if documentMode: print("Score train:", Score_train)
    if documentMode: print("Score test:", Score_test)
    #if documentMode: print("Classifier Predictions:", predictions)
    if documentMode: print("Confusion Matrix: tn, fp, fn, tp", Confusion_matrix)
    if documentMode: print("true positive rate:", tpr)
    if documentMode: print("false positive rate:", fpr)
    
    return [classifier_model,
            #Score_train,
            Score_test,
            list(data_frame_id_exists),
            list(predictions),
            Confusion_matrix,
            fpr, tpr
           ]


#Hier können Parameter übergeben werden: Welcher Coin und auf was wird klassifiziert
classifier_result=cryptoclassifier('bitcoin','close') 



Score test: 0.5777777777777777
Confusion Matrix: tn, fp, fn, tp [[217 596]
 [160 820]]
true positive rate: [0.         0.83673469 1.        ]
false positive rate: [0.         0.73308733 1.        ]


In [ ]:
list_AdaBoost = []

zaehler=0
for current_run in range(10):
    list_regression.append(
        {
            'Score Test:' cryptoclassifier()
            'True Positive Rate': cryptoclassifier(current_classifier,'close')[5] 
            'Score Test': cryptoclassifier(current_classifier,'close')[3] 
        }) #fuege ergebnis zur Ergebnistabelle hinzu
    zaehler=zaehler+1

In [34]:
#Liste aller classifier
classifier = [AdaBoostClassifier,
              RandomForestClassifier
    
]

#Iteriere über verschiedene n_estimators und learning rates
list_classifier = []
zaehler=0
for current_classifier in classifier:
    list_classifier.append(
        {
            'name':current_classifier
            'True Positive Rate': cryptoclassifier(current_classifier,'close')[5] 
            'Score Test': cryptoclassifier(current_classifier,'close')[3] 
        }) #fuege ergebnis zur Ergebnistabelle hinzu
    zaehler=zaehler+1
    print("regressed Number ", zaehler, " von ", len(different_coins))
    if zaehler == 1300: break
    
zaehler= 0
df_regression_result=pd.DataFrame(list_regression)
print(df_regression_result)

SyntaxError: invalid syntax (<ipython-input-34-cfeb0471b688>, line 14)

In [36]:
Y = np.random.randint(2, size=1793)
Y

array([1, 1, 1, ..., 0, 0, 0])